In [23]:
import requests
import pandas as pd
import numpy as np
import soilgrids

In [30]:
def get_soil_data (lat:int, lon:int):
    url=f"https://rest.isric.org/soilgrids/v2.0/properties/query?lon={lon}&lat={lat}&property=bdod&property=cec&property=cfvo&property=clay&property=nitrogen&property=ocd&property=ocs&property=phh2o&property=sand&property=silt&property=soc&depth=0-5cm&depth=0-30cm&depth=5-15cm&depth=15-30cm&depth=30-60cm&depth=60-100cm&depth=100-200cm"    
    response = requests.get(url)
    return response.json()
    

In [31]:
def process_soil_dictionary (soil:dict): 
    resp=soil['properties']['layers']
    result={}
    for feature in resp:
        for depth_range in feature['depths']:
            result [f"{feature['name']}_{depth_range['range']['top_depth']}_{depth_range['range']['bottom_depth']}"]=depth_range.get('values').get('mean',np.nan)
    result=pd.DataFrame(result, index=[0])
    return result
    


In [36]:
def return_soil_features (lat, lon):
    return process_soil_dictionary(get_soil_data(lat,lon))

In [37]:
return_soil_features (-9, -72)

,bdod_0_5,bdod_5_15,bdod_15_30,bdod_30_60,bdod_60_100,bdod_100_200,cec_0_5,cec_5_15,cec_15_30,cec_30_60,...,silt_15_30,silt_30_60,silt_60_100,silt_100_200,soc_0_5,soc_5_15,soc_15_30,soc_30_60,soc_60_100,soc_100_200
0,116,124,129,138,139,141,152,117,113,118,...,422,364,355,375,272,150,89,56,38,34
